In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

## Funzioni di trasferimento

In [ ]:
def RC_R(R, C, W):
    mod = R/(np.sqrt(R**2 + 1/(W*C)**2))
    fase = np.arctan(1/(W*C*R))

    return mod, fase

def RC_C(R, C, W):
    mod = 1/np.sqrt(1+(R*C*W)**2)
    fase = np.arctan(R*C*W)

    return mod, fase

def RL_R(R, L, W):
    mod = R/(R**2 + (W*L)**2)
    fase = np.arctan(W*L/R)

    return mod, fase

def RL_L(R, L, W):
    mod = W*L/(R**2 + (W*L)**2)
    fase = np.arctan(W*L/R)

def RLC_R(R,C,L,W):
    mod = R/(np.sqrt((R**2)+((W*L)-(1/(W*C))**2)))
    fase = np.arctan((((W**2)*L*C)-1)/(W*C*R))
    
    return mod, fase

def RLC_C(R,C,L,W):
    mod =  1/(np.sqrt((1-((W**2)*L*C))**2)+((W**2)*(C**2)*(R**2)))
    fase = np.arctan((W*C*R)/(1-((W**2)*L*C)))
    
    return mod, fase


def RLC_L(R,C,L,W):
    mod = ((W**2)*(L**2))/(np.sqrt((R**2) + ((W*L)-(1/(W*C)))**2))
    fase = np.arctan((((W**2)*L*C)-1)/(W*C*R))

    return mod, fase

# Prima parte RC RL 

# CIRCUITO RC
 Capacità utilizzata= $99 \pm 1$ nF \
 Resistenza= $100\pm 1 \Omega$ (check)
 
 Incertezza fase (CH2-CH1) = $\pm 1$ \
 Incertezza Ampiezza V_in(CH1) = $\pm 0.05$   Trasportarla su asse x e asse y \
 Incertezza Ampiezza V_out (CH2) = $\pm 0.05$ \
 Incertezza differenza (MATH)  = $\pm 0.01$ \

 le fasi nel foglio sono scritte in gradi

In [ ]:
dati1 = pd.read_excel("dati_rc.xlsx")

RC_freq = dati1["Frequenza"]
RC_amp_A = dati1["Ampiezza Va"]
RC_amp_B = dati1["Ampiezza Vb"]
RC_amp_ab = dati1["Ampiezza Va-b"]
RC_fase_A = dati1["ddf a/b"] 
RC_fase_AB = dati1["ddf a-b/a"]

In [ ]:
# PLot dati RC resitenza

plt.plot(RC_freq, [i/j for i,j in zip(RC_amp_B, RC_amp_A)])
plt.scatter(RC_freq, [i/j for i,j in zip(RC_amp_B, RC_amp_A)], color ='r')
plt.xscale('log')
plt.grid()
plt.show()

In [ ]:
#plot dati capacità

plt.plot(RC_freq, [i/j for i,j in zip(RC_amp_ab, RC_amp_A)])
plt.scatter(RC_freq, [i/j for i,j in zip(RC_amp_ab, RC_amp_A)], color ='r')
plt.xscale('log')
plt.grid()
plt.show()

# CIRCUITO RL


Induttanza utilizzata =   \
Resistenza utilizzata =   \
Incertezza fase =   \
Incertezza ampiezza V_in =  \ 
Incertezza ampiezza V_out =  \
Incertezza differenza (MATH) =   


# CIRCUITO RLC

Resistenza utilizzata = 
Induttanza utilizzata = 
Capacità utilizzata = 

